In [ ]:
# %pip uninstall pymilvus==2.4.13
# %pip install pymilvus==2.4.13




In [1]:
from pymilvus import connections

try:
    connections.connect(alias="default", host="127.0.0.1", port="19530")
    print("Connection successful!")
except Exception as e:
    print(f"Connection failed: {e}")


Connection successful!


In [23]:
elastic_ip = '34.0.64.248:9200'
kibana_ip = '34.0.64.248:5601'
es_username = 'user'
es_password = 'knesset'



# ! pip install elasticsearch

Getting the relevent results from elastic

In [14]:
from elasticsearch import Elasticsearch

es = Elasticsearch(f'http://{elastic_ip}', basic_auth=(es_username, es_password), request_timeout=100000)
data_q =[]
# Query definition
query = {
    "query": {
        "bool": {
            "must": [
                {"match": {"speaker_name": "מירי רגב"}}
            ],
            "filter": {
                "script": {
                    "script": {
                        "source": "doc['sentence_text.keyword'].size() > 0 "}
                }
            }
        }
    }
}

# Initialize scroll
resp = es.search(index="all_features_sentences", body=query, scroll="2m", size=1000)

# Retrieve the scroll ID and first batch of hits
scroll_id = resp['_scroll_id']
hits = resp['hits']['hits']

total_hits = 0
while total_hits<5000:
    for hit in hits:
      data_q.append("%(sentence_text)s" % hit["_source"])
      #print("%(sentence_text)s" % hit["_source"])

    total_hits += len(hits)

    # Fetch the next batch
    resp = es.scroll(scroll_id=scroll_id, scroll="2m")
    scroll_id = resp['_scroll_id']
    hits = resp['hits']['hits']

print(f"Total results retrieved: {total_hits}")

# Clear the scroll to free resources
es.clear_scroll(scroll_id=scroll_id)


C:\Users\orifl\AppData\Local\Temp\ipykernel_11804\1500901426.py:23: DeprecationWarning: Received 'size' via a specific parameter in the presence of a 'body' parameter, which is deprecated and will be removed in a future version. Instead, use only 'body' or only specific parameters.
  resp = es.search(index="all_features_sentences", body=query, scroll="2m", size=1000)


ConnectionTimeout: Connection timed out

Cleaning the text

In [4]:
import re

data_q_filtered = []

for sentence in data_q:
    # Keep only Hebrew letters, numbers and spaces
    cleaned_sentence = re.sub(r'[^א-ת0-9 %]', '', sentence).strip()
    # Check word count
    if len(cleaned_sentence.split()) >= 5:
        data_q_filtered.append(cleaned_sentence)
# docs = data_q_filtered.copy()


Creating the schema 

In [6]:
from pymilvus import FieldSchema, CollectionSchema, DataType, Collection

fields = [
    FieldSchema(name="id", dtype=DataType.INT64, is_primary=True),
    FieldSchema(name="text", dtype=DataType.VARCHAR, max_length=512),
    FieldSchema(name="vector_minilm", dtype=DataType.FLOAT_VECTOR, dim=384),
    FieldSchema(name="vector_bert", dtype=DataType.FLOAT_VECTOR, dim=1024),
    FieldSchema(name="vector_e5", dtype=DataType.FLOAT_VECTOR, dim=768),
]

schema = CollectionSchema(fields, description="3Modelcomparison_test")
collection_name = "threeModelcomparison_test211"
collection = Collection(name=collection_name, schema=schema)


Using the embeddings models 

In [ ]:
from sentence_transformers import SentenceTransformer

# Initialize models
minilm_model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")
e5_model = SentenceTransformer("intfloat/multilingual-e5-large")
bert_model = SentenceTransformer("bert-base-nli-stsb-mean-tokens")  # BERT-based model

# Encode with each model
vectors_minilm = minilm_model.encode(flattened_docs)  # Dim: 384
vectors_e5 = e5_model.encode(flattened_docs)          # Dim: 1024
vectors_bert = bert_model.encode(flattened_docs)      # Dimension depends on the model (768 for standard BERT)

# Prepare data
data = [
    {
        "id": i,
        "text": flattened_docs[i],
        "vector_minilm": vectors_minilm[i],
        "vector_e5": vectors_e5[i],
        "vector_bert": vectors_bert[i],
    }
    for i in range(len(flattened_docs))
]

# Milvus expects columns rather than rows
field_data = [
    [item["id"] for item in data],
    [item["text"] for item in data],
    [item["vector_minilm"] for item in data],
    [item["vector_e5"] for item in data],
    [item["vector_bert"] for item in data],
]

# Insert
res = collection.insert(field_data)
print(f"Inserted {len(docs)} rows.")


c:\Users\orifl\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


TypeError: SentenceTransformer.encode() missing 1 required positional argument: 'sentences'

Creating the indexes

In [24]:
index_params_minilm = {
    "index_type": "IVF_FLAT",
    "metric_type": "COSINE",
    "params": {"nlist": 128},
}
index_params_e5 = {
    "index_type": "IVF_FLAT",
    "metric_type": "COSINE",
    "params": {"nlist": 128},
}
index_params_bert = {
    "index_type": "IVF_FLAT",
    "metric_type": "COSINE",
    "params": {"nlist": 128},
}

collection.create_index(field_name="vector_minilm", index_params=index_params_minilm)
collection.create_index(field_name="vector_e5",     index_params=index_params_e5)
collection.create_index(field_name="vector_bert",     index_params=index_params_bert)
print("Indexes created.")

# Load collection
collection.load()


Indexes created.


In [25]:
# Example: compare the first 'Culture' quote to others
quote_idx = 0  # index in flattened_docs
test_vector = vectors_minilm[quote_idx]

# Suppose you do a search in Milvus or simply measure cosine similarity
# with each embedding in Python
import numpy as np
from numpy.linalg import norm

similarities = []
for i, vec in enumerate(vectors_minilm):
    cos_sim = np.dot(test_vector, vec) / (norm(test_vector) * norm(vec))
    similarities.append((i, cos_sim))

# Sort by descending similarity
similarities.sort(key=lambda x: x[1], reverse=True)

# Print top 5 most similar
print("Top 5 most similar quotes (using MiniLM embeddings):")
for idx, sim in similarities[:5]:
    print(f"- Index {idx} (Group {group_ids[idx]}), Similarity: {sim:.3f}, Text: {flattened_docs[idx]}")


Top 5 most similar quotes (using MiniLM embeddings):
- Index 0 (Group 0), Similarity: 1.000, Text: אני מאמינה שהתרבות צריכה לשרת את הציבור ולא להתרחק ממנו.
- Index 6 (Group 0), Similarity: 0.816, Text: חשוב שהתרבות תשקף את הקול של האזרחים ולא תימלט ממנו.
- Index 1 (Group 0), Similarity: 0.788, Text: התרבות חייבת להישאר נגישה לאנשים, לא להתנשא מעליהם.
- Index 3 (Group 0), Similarity: 0.773, Text: על התרבות להתחבר לעם במקום לבודד את עצמה במגדל שן.
- Index 19 (Group 0), Similarity: 0.762, Text: המטרה שלנו היא לקדם תרבות שמייצגת את העם ולא מנתקת אותו.


: 

Quering the indexes to compare the resutls

In [21]:
# Single user query
queries = ["איך יום ראשון בעבודה נראה?",
           "מה הכי טעים לאכול"] #add the query here
# Encode all 10 queries with both models
query_vectors_minilm = minilm_model.encode(queries)  # shape: (10, 384)
query_vectors_e5 = e5_model.encode(queries)          # shape: (10, 1024)

search_params = {"metric_type": "COSINE", "params": {"nprobe": 10}}
limit = 5  # get top 5 results per query


# Search using MiniLM embeddings
results_minilm = collection.search(
    data=query_vectors_minilm.tolist(),
    anns_field="vector_minilm",
    param=search_params,
    limit=limit,
    output_fields=["text"],
)
print("\n--- Results from MiniLM model ---")
for result in results_minilm:
    for hit in result:
        print(f"Text: {hit.entity.get('text')} (Score: {hit.score})")

# Search using E5 embeddings
results_e5 = collection.search(
    data=query_vectors_e5.tolist(),
    anns_field="vector_e5",
    param=search_params,
    limit=limit,
    output_fields=["text"],
)
print("\n--- Results from E5 model ---")
for result in results_e5:
    for hit in result:
        print(f"Text: {hit.entity.get('text')} (Score: {hit.score})")



--- Results from MiniLM model ---
Text: והלאכשרים האלה של מתן כהנא שעה לפני כניסת שבת (Score: 0.5714821219444275)
Text: ואת יום השבת  יגידו שהוא יום בחירה כי היום במדינה שהיא כלל אזרחיה זה יום ראשון לא יום שבת ויבטלו את יום השבת (Score: 0.5456657409667969)
Text: בואו נקווה ונשמע מה תהיה ההחלטה שלהם ביום ראשון (Score: 0.5010085701942444)
Text: אילו לא היו מפטרים את יום טוב ערב יום שישי  דיינו (Score: 0.48870089650154114)
Text: בעוד כמה ימים נחגוג את חג השבועות חג מתן תורה (Score: 0.4839223027229309)
Text: מה אני רוצה לאכול בשבת (Score: 0.6869887113571167)
Text: בואי נשאל את אנשי  מה הם רוצים לאכול לשבת נתאם איתם (Score: 0.6330589056015015)
Text: גם את הבשר שלכם הוא יאכל (Score: 0.5654752254486084)
Text: לא אוכלים תוך כדי לא אוכלים (Score: 0.5333347916603088)
Text: הרי ברור שאת סטייק הפילה ואת הקוויאר לא תעלו כי מי אוכל את העוף והדגים (Score: 0.5321474671363831)

--- Results from E5 model ---
Text: צריך למנוע תאונות עבודה לכל אחד (Score: 0.8441827297210693)
Text: אז על מי באתם לעבוד (Sc

sentence transformers embading 

In [8]:
# %pip install sentence_transformers

from sentence_transformers import SentenceTransformer

# Initialize the embedding model
embedding_sentence_tranformes = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

# Encode the documents
vectors_sentences_transformes = embedding_sentence_tranformes.encode(docs)  # Encode into vectors
print("Dim:", len(vectors_sentences_transformes[0]), vectors_sentences_transformes[0].shape)  # Dim: 768


In [7]:
data = [
    {"id": i, "vector": vectors_sentences_transformes[i], "text": docs[i]}
    for i in range(len(vectors_sentences_transformes))
]
print("Data has", len(data), "entities, each with fields:", data[0].keys())
print("Vector dim:", len(data[0]["vector"]))


Data has 3308 entities, each with fields: dict_keys(['id', 'vector', 'text'])
Vector dim: 384


intfloat/multilingual-e5-large

In [121]:
# %pip install transformers
# %pip install torch

In [19]:
from transformers import AutoTokenizer, AutoModel
import torch
e5_tokenizer = AutoTokenizer.from_pretrained("intfloat/multilingual-e5-large")
e5_model = AutoModel.from_pretrained("intfloat/multilingual-e5-large")
inputs = e5_tokenizer(docs, return_tensors="pt", padding=True, truncation=True)

# Generate embeddings
with torch.no_grad():
    e5_embeddings = e5_model(**inputs).last_hidden_state.mean(dim=1)

print(e5_embeddings.shape)  # Shape: (number_of_texts, embedding_dimension)


KeyboardInterrupt: 

In [12]:
data = [
    {"id": i, "vector": e5_embeddings[i], "text": docs[i]}
    for i in range(len(e5_embeddings))
]
print("Data has", len(data), "entities, each with fields:", data[0].keys())
print("Vector dim:", len(data[0]["vector"]))


Data has 3308 entities, each with fields: dict_keys(['id', 'vector', 'text'])
Vector dim: 1024


In [13]:
from pymilvus import FieldSchema, CollectionSchema, DataType, Collection

# Define the collection schema
fields = [
    FieldSchema(name="id", dtype=DataType.INT64, is_primary=True),
    FieldSchema(name="vector", dtype=DataType.FLOAT_VECTOR, dim=len(e5_embeddings[0])),
    FieldSchema(name="text", dtype=DataType.VARCHAR, max_length=1024),
]
schema = CollectionSchema(fields, description="Demo collection for quotes of Miri Regev")

# Create or load the collection
collection_name = "Miri_Regev_quotes_e5"
collection = Collection(name=collection_name, schema=schema)


In [14]:
# Milvus expects the data in columns (field-wise)
field_data = [
    [entity["id"] for entity in data],  # IDs
    [entity["vector"] for entity in data],  # Vectors
    [entity["text"] for entity in data],  # Text
]

# Insert the data
res = collection.insert(field_data)
print(f"Inserted {len(field_data[0])} entities into {collection_name}.")


Inserted 3308 entities into Miri_Regev_quotes_e5.


In [15]:
index_params = {
    "index_type": "IVF_FLAT",
    "metric_type": "L2",
    "params": {"nlist": 128},
}
collection.create_index(field_name="vector", index_params=index_params)
print("Index created.")


Index created.


In [16]:
from pymilvus import list_collections

collections = list_collections()
print("Collections in Milvus:", collections)




Collections in Milvus: ['Miri_Regev_quotes_e5']


In [94]:
collection = Collection(name="Miri_Regev_quotes_longer")


In [18]:
collection.load()

# Define a query vector (e.g., embedding for a new query)
query_vector = embedding_fn.encode(["חינם נסיעה?"])[0]

# Perform the search
search_params = {"metric_type": "L2", "params": {"nprobe": 10}}
results = collection.search(
    data=[query_vector],  # Query vector
    anns_field="vector",
    param=search_params,
    limit=15,  # Number of top matches
    output_fields=["text"],  # Include the text field in results
)

# Display results
for result in results:
    for hit in result:
        print(f"Text: {hit.entity.get('text')}")




AttributeError: 'Tensor' object has no attribute 'encode'